In [1]:
# !pip install --force-reinstall torch==1.13.1+cu117 --index-url https://download.pytorch.org/whl/cu117

In [2]:
import os
# import constants
# from datasets import load_dataset
# from utils import clean, make_current_datetime_dir, compute_metrics, preprocess_data

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import TrainingArguments, Trainer, logging

import torch

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from tensorboard.backend.event_processing import event_accumulator

2024-05-07 16:48:36.226365: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 16:48:36.226421: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 16:48:36.226463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 16:48:36.235993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

# CUDA_DEVICE_ORDER 및 CUDA_VISIBLE_DEVICES 환경 변수 확인
cuda_order = os.getenv("CUDA_DEVICE_ORDER")
cuda_visible_devices = os.getenv("CUDA_VISIBLE_DEVICES")

print("CUDA_DEVICE_ORDER:", cuda_order)
print("CUDA_VISIBLE_DEVICES:", cuda_visible_devices)

print(torch.__version__)

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


CUDA_DEVICE_ORDER: PCI_BUS_ID
CUDA_VISIBLE_DEVICES: 5
1.13.1+cu117
Using cuda device


In [4]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

1.13.1+cu117
True
0
Tesla V100-PCIE-32GB


In [5]:
mydata = 'mydata/'
train_path = mydata+'mydata_train.csv'
test_path = mydata+'mydata_test.csv'
valid_path = mydata+'mydata_valid.csv'

In [6]:
# 데이터 불러오기
train_data = pd.read_csv(train_path, sep=',')
test_data = pd.read_csv(test_path, sep=',')
valid_data = pd.read_csv(valid_path, sep=',')

# 컬럼 이름 설정 (예시)
train_data.columns = ['text', 'label']
test_data.columns = ['text', 'label']
valid_data.columns = ['text', 'label']

In [7]:
# processed_train_data = preprocess_data(train_data)
# unq_labels, cnt_labels = np.unique(processed_train_data['label'].values, return_counts=True)
# print('labels =', dict(zip(unq_labels, cnt_labels)))
# filtered_data = processed_train_data[processed_train_data['label'] == 0]
# filtered_data
# processed_train_data
# train_data

In [8]:
from transformers import AutoTokenizer, AutoModel


tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
model = AutoModelForSequenceClassification.from_pretrained('beomi/KcELECTRA-base-v2022', num_labels=3)
model = model.to(device)
# print(next(model.parameters()).device)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
max_len = 64          # 문장의 길이 (평균 13정도)
batch_size = 32
num_epochs = 2
log_interval = 500    # metrics 생성 시점

# 학습 데이터
# ==> 토크나이징 + 패딩 : dim=64
encoded_train = tokenizer(
    train_data['text'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 확인
print( encoded_train[0].tokens )
print( encoded_train[0].ids )
print( encoded_train[0].attention_mask )
print()
print('디코딩 :',tokenizer.decode(encoded_train[0].ids))

['[CLS]', '자한당', '##틀딱', '##들', '.', '.', '악플', '##질', '고만해라', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[2, 8535, 15271, 4079, 18, 18, 10377, 4077, 27100, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

디코딩 : [CLS] 자한당틀딱들.. 악플질 고만해라. [SEP

In [10]:
# 검증 데이터
# ==> 토크나이징 + 패딩 : dim=64
encoded_valid = tokenizer(
    valid_data['text'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print('디코딩 :',tokenizer.decode(encoded_valid[0].ids))
print('디코딩 :',tokenizer.decode(encoded_valid[-1].ids))

디코딩 : [CLS] 서로만이 이렇게 있을때 보기 좋다♥ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
디코딩 : [CLS] 아근데 진짜 영화가 너무 웃김 ㅋㅋㅋㅋㅋㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [11]:
# 학습 시작 전에 label 데이터가 int형에 1차원인지 확인

print(train_data['label'].values.shape, train_data['label'].values.dtype)
print(valid_data['label'].values.shape, valid_data['label'].values.dtype, end='\n\n')
train_data['label'].values[0]

(78977,) int64
(8776,) int64



1

In [12]:
class KoelectraDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [13]:
# 데이터셋 생성
train_dataset = KoelectraDataset(encoded_train, train_data['label'].values)
valid_dataset = KoelectraDataset(encoded_valid, valid_data['label'].values)



In [14]:
#정확도 측정을 위한 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # average: 'micro', 'macro', 'weighted' or 'samples' 
    # 참고 https://aimb.tistory.com/152
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'acc': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [15]:
# 성능 시각화 wandb
# https://docs.wandb.ai/guides/integrations/huggingface

# 학습 파라미터
training_args = TrainingArguments(
    output_dir='./temp/electra',
    overwrite_output_dir='True',

    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
 
    logging_dir='./temp/logs',
    logging_steps=log_interval,
    evaluation_strategy="steps",
    eval_steps=log_interval,

    # https://discuss.huggingface.co/t/save-only-best-model-in-trainer/8442/8
    save_total_limit=2,
    save_strategy='no',
    load_best_model_at_end=False,
    
    warmup_ratio=0.1

)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall
500,0.576800,0.372312,0.865428,0.867787,0.877032,0.865428
1000,0.324800,0.298580,0.890269,0.889518,0.889639,0.890269
1500,0.293400,0.325407,0.882293,0.883476,0.889557,0.882293
2000,0.270200,0.261171,0.904740,0.904584,0.904483,0.904740
2500,0.263100,0.294268,0.903373,0.902853,0.902832,0.903373
3000,0.188600,0.279909,0.904512,0.904577,0.904708,0.904512
3500,0.175800,0.298261,0.908387,0.908326,0.908283,0.908387
4000,0.170800,0.290942,0.908273,0.908426,0.908661,0.908273
4500,0.159500,0.284275,0.910665,0.910714,0.910827,0.910665


TrainOutput(global_step=4938, training_loss=0.26043323410238123, metrics={'train_runtime': 753.4933, 'train_samples_per_second': 209.629, 'train_steps_per_second': 6.553, 'total_flos': 5194977097976064.0, 'train_loss': 0.26043323410238123, 'epoch': 2.0})

In [17]:
trainer.save_model()